In [5]:
### README ###
'''
This is my first DS project in which I developed neural network model which can predict the price of used bicicles (road or mtb)
I used two datasets found on kaggle (linki) which included data scraped from two websites, 
bike-exchange.com - this dataset was well prepered, with few amount of columns wich contained valuable data
ebay.com - dataset included 500 columns which required a lot of struggle to compress the data to 10 cathegories 

There were 3 significant stages of my project:
- cleaning and preprocessing the data: 
    - merging 2 datasets into one (cleaning/preprocessing functions which I created universal and works god for both datasets)
    - grouping types of bike, gropsets, brakes, suspension and extracting year of production, frame material, condition and wheel size  
    - filling nans by median in very few instances, where it was safe 
    - mapping cathegorical data to numeric, I used linear scale for the most columns, although year production had a wide tail, which was flattened with QuantileTransformer from sklearn
    - as a result two datasets were made, numerical and cathegorical (one hot encoding)
- training model
    - 
'''

'\nThis is my first DS project in which I developed neural network model which can predict the price of used bicicles (road or mtb)\nI used two datasets found on kaggle (linki) which included data scraped from two websites, \nebay.com, bike-exchange.com.\nThere were 3 significant stages of my project:\n- cleaning the data \n\n- \n'

In [ ]:
import pandas as pd 
import re
import numpy as np

df_exch = pd.read_json('data\\raw_sources\data_bike_exchange.json', lines=True)
df_ebay = pd.read_json('data\\raw_sources\data_ebay.json', lines=True)

df = pd.concat([df_exch, df_ebay], axis = 0).reset_index(drop=True)
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df

In [ ]:
def print_nans_perc(df : (pd.DataFrame, pd.Series), p = False, perc = 0.95, sort_by = 'val'):
    n = len(df)
    nans_perc = df.isna().sum() / n
    print('len - ', n)
    if type(df) == pd.DataFrame:
        if sort_by == 'val':
            nans_perc.sort_values(ascending=True, inplace=True)
        else:
            nans_perc.sort_index(inplace = True)
        if p:
            for index, val in nans_perc.items():
                print(index + ': ', val)
            print('\n')
        cols = nans_perc[nans_perc > 0.95].index
        return cols
    elif type(df) == pd.Series:
        if p:
            print('left_nans: ', nans_perc)
    return
print_nans_perc(df, p = True)

In [ ]:
def drop_nans_in_cols(df : pd.DataFrame, cols : list):
    print(len(df))
    df.dropna(subset=cols, inplace=True)
    df.dropna(axis = 1, how = 'all', inplace=True)
    print(len(df))
    print_nans_perc(df)

In [ ]:
road_groups = ['road bike', 'road bike - racing', 'road bike', 'road bike - touring', 'road', 'cyclocross bike', 'cyclocross/gravel', 'road disc bike', 'road bike - racing or touring', 
            'aero road bike', 'racing/touring roadbike', 'road/touring', 'time trial/triathlon bike', 'road bike racing', 'time trial/triathlon', 'road/racing', 'racing/road bike', 
            'raod bike', 'road racing bike', 'road bike', 'road bike - touring ??', 'road bike / triathlon', 'racing/road bike', 'comfort road bike',  'road bike', 'road bike / racing bike', 'hybrid', 
            'gravel, cx, road, commuter', 'road bike / cyclocross bike / triathlon', 'race road bike', 'road bike - racing', 'gravel bike', 'road bike - touring', 'racing bike', 'road bike racing', "road", 'road bike', 'road bikes',]

mtb_groups = ['mountain bike', 'full suspension', 'downhill bike', 'mountain bike', 'mountain bike/ gravel', 'mountain bike mtb 29er', 'mountain suspension bike', 'hardtail bike','mountain bike', 'mountain  bike style', 
            'hardtail mountain bike', 'mountain', 'mountain bike', 'mountain bike19"', 'enduro', '29er mountain bike', 'mountain / jump bike', 'mountain bike mtb 29er', 'hardtail bike', 'downhill bike', 'full suspension mountain bikes', 
            'hardtail mountain bikes', 'rigid mountain bikes', 'mountain bikes', 'mtb', 'mountain', 'fat bike']

types_dict = {'road bike': 'road bike',  'road bike - racing': 'road bike',  'road bike': 'road bike',  'road bike - touring': 'road bike',  'road': 'road bike',  'cyclocross bike': 'road bike',  'cyclocross/gravel': 'road bike',  
            'road disc bike': 'road bike',  'road bike - racing or touring': 'road bike',  'aero road bike': 'road bike',  'racing/touring roadbike': 'road bike',  'road/touring': 'road bike',  'time trial/triathlon bike': 'road bike', 
            'road bike racing': 'road bike',  'time trial/triathlon': 'road bike',  'road/racing': 'road bike',  'racing/road bike': 'road bike',  'raod bike': 'road bike',  'road racing bike': 'road bike',  'road bike': 'road bike',  
            'road bike - touring ??': 'road bike',  'road bike / triathlon': 'road bike',  'racing/road bike': 'road bike',  'comfort road bike': 'road bike',   'road bike': 'road bike',  'road bike / racing bike': 'road bike',  'hybrid': 'road bike',  
            'gravel, cx, road, commuter': 'road bike',  'road bike / cyclocross bike / triathlon': 'road bike',  'race road bike': 'road bike',  'road bike - racing': 'road bike',  'gravel bike': 'road bike',  'road bike - touring': 'road bike',  
            'racing bike': 'road bike',  'road bike racing': 'road bike',  "road": 'road bike', 'road bike': 'road bike',  'road bikes': 'road bike', 'mountain bike': 'mtb',  'full suspension': 'full',  'downhill bike': 'full',  'mountain bike': 'mtb',  
            'mountain bike/ gravel': 'mtb',  'mountain bike mtb 29er': 'mtb',  'mountain suspension bike': 'mtb',  'hardtail bike': 'hard', 'mountain bike': 'mtb',  'mountain  bike style': 'mtb',  'hardtail mountain bike': 'hard',  'mountain': 'mtb',  
            'mountain bike': 'mtb',  'mountain bike19"': 'mtb',  'enduro': 'full',  '29er mountain bike': 'mtb',  'mountain / jump bike': 'mtb',  'mountain bike mtb 29er': 'mtb',  'hardtail bike': 'hard',  'downhill bike': 'full',  
            'full suspension mountain bikes': 'full',  'hardtail mountain bikes': 'hard',  'rigid mountain bikes': 'mtb',  'mountain bikes': 'mtb', 'mountain': 'mtb', 'fat bike': 'mtb'}

def find_from_list(row, columns, list):
    for col in columns:
        value = row[col]
        found = ''
        if pd.notna(value) and isinstance(value, str): 
            found = re.findall(r'(' + '|'.join(list) + ')', value, re.IGNORECASE)
        if found:
            return str(found[0]).lower()
    return np.nan

def make_num_cat(df_orig : pd.DataFrame, col_name : str, cat_list : list, dictonary: dict, find_cols = [], find_fn = find_from_list, num_type = float):
    if find_cols == []:
        find_cols = df_orig.columns
    cat_col = col_name + '_cat'
    num_col = col_name + '_num'
    df = df_orig.copy()
    df[[cat_col, num_col]] = np.nan
    df[cat_col] = df.apply(find_fn, axis = 1, args = (find_cols, cat_list))
    print_nans_perc(df[cat_col], p = True)
    df[num_col] = df[cat_col].map(dictonary)
    df[num_col] = df[num_col].astype(num_type)
    return df

cols_ty = ['Bike Type', 'Type', 'Title', 'Seller notes', 'Product URL']
df = make_num_cat(df, 'type', road_groups + mtb_groups, types_dict, cols_ty, num_type=str)
df['type'] = df['type_num']
df.drop(columns=['type_num', 'type_cat'], inplace=True)
print_nans_perc(df['type'], p=1)
df.groupby('type').size()

In [ ]:
df['Price'] = df['Price'].combine_first(df['Price now'])
df['Price'].replace('[A-Za-z, \$, £]','',inplace=True, regex=True)
df = df[df['Price'] != '']
df['Price'] = df['Price'].astype(float)
print_nans_perc(df['Price'], p=1)

In [ ]:
gear_to_group_dict_road = {
        '1990 2003': {
               8.0: 'sora',
               9.0: 'ultegra',
        },
        '2004 2008': {
               8.0:'sora',
               9.0: '105',           
               10.0: 'dura ace',
            },
        '2009 2013': {
               6.0: 'tourney',
               8.0: 'sora',   
               9.0: 'deore',
               10.0: '105',
               11.0: 'ultegra',
        },
        '2014 2016': {
               7.0: 'tourney',
               8.0: 'claris',
               9.0: 'sora',
               10.0: '105',
               11.0: 'ultegra',
        },
        '2017 2023': {
               5.0: 'tourney',
               6.0: 'tourney',
               7.0: 'tourney',
               8.0: 'claris',
               9.0: 'sora',
               10.0: '105',
               11.0: 'ultegra',
               12.0: 'dura ace',
          }         
     }
gear_to_group_dict_mtb = {
          '1990 2003': {
               7.0:'altus',
               8.0: 'deore',
               9.0:'xtr',
               },
          '2004 2008':{
               8.0: 'altus',
               9.0: 'deore xt',
               10.0:'xtr',
          },
          '2009 2013': {
               6.0: 'tourney',
               8.0: 'altus',
               9.0: 'deore',
               10.0: 'xtr',
               11.0: 'ultegra',
          },
          '2014 2016': {
               7.0: 'tourney',
               8.0: 'tourney',
               9.0: 'altus',
               10.0: 'deore xtr',
               11.0: 'xtr',
          },
          '2017 2023': {
               5.0: 'tourney',
               6.0: 'tourney',
               7.0: 'tourney',
               8.0: 'tourney',
               9.0:'altus',
               10.0: 'deore',
               11.0: 'deore xt',
               12.0:'xtr',
          }
     }

bike_groups = [
    "Dura Ace", "Dura-Ace", "Ultegra", "105", "Tiagra", "Sora", "Claris", "GRX",
    
    "XTR", "Deore XT", "SLX", "Deore", "Alivio", "Acera", "Altus", "Tourney", "Zee", "Saint",
    
    "Super Record", "Record", "Chorus", "Potenza", "Centaur", "Veloce", "Athena",
    
    "Sram Red", "Force", "Rival", "Apex",

    "XX1", "X01","GX", "NX", "SX", "XX", "X0", "X9", "X7", "X5", "Descendant", "Code"
]

road_groups = [ "dura ace", "dura-ace", "ultegra", "105", "tiagra", "sora", "claris", "grx",
               "super record", "record", "chorus", "potenza", "centaur", "veloce", "athena",
                "sram red", "Force", "arval", "apex"]

mtb_groups = [
    "xtr", "deore xt", "slx", "deore", "alivio", "acera", "altus", "tourney", "zee", "saint",
    "xx1", "x01", "gx", "nx", "sx", "xx", "x0", "x9", "x7", "x5", "descendant", "code"
]

bike_groups_dict = {
    "dura ace": 1.0, "dura-ace": 1.0, "ultegra": 0.9, "105": 0.8, "tiagra": 0.7, "sora": 0.6, "claris": 0.5, "grx": 0.8,
    
    "xtr": 1.0, "deore xt": 0.9, "slx": 0.8, "deore": 0.7, "alivio": 0.6, "acera": 0.5, "altus": 0.4, "tourney": 0.3, "zee": 0.8, "saint": 0.9,
    
    "super record": 1.0, "record": 0.9, "chorus": 0.8, "potenza": 0.7, "centaur": 0.6, "veloce": 0.5, "athena": 0.4,
    
    "sram red": 1.0, "force": 0.9, "rival": 0.8, "apex": 0.7,
    
    "xx1": 1.0, "x01": 0.9, "gx": 0.8, "nx": 0.7, "sx": 0.6, "xx": 0.9, "x0": 0.8, "x9": 0.7, "x7": 0.6, "x5": 0.5, "descendant": 0.7, "code": 0.7
}


df.drop(['Color', 'Colour', 'Product URL', 'Photo URL'], axis=1, inplace=True)

In [ ]:
def find_year(row, cols):
    for col in cols:
        value = row[col]
        if pd.notna(value) and isinstance(value, str):
            four_num = re.findall(r'\b[0-9]{4}\b', value)
            four_num = [float(num) for num in four_num]
            for num in four_num:
                if 1950 <= num <= 2024:
                    return num
    return np.nan

df['Production_year'] = np.nan
cols = ['Model Year','Title'] 
#cols = df.columns
df['Production_year'] = df.apply(find_year, axis=1, args=(cols,))
print_nans_perc(df['Production_year'], p=True)

In [ ]:
cols_gp = ['Title', 'Seller notes', 'Model', 'Brake Type', 'Gear Change Mechanism', 'Groupset', 'Rear Derailleur']
df = make_num_cat(df, 'groupset', bike_groups, bike_groups_dict, cols_gp)
df['groupset_cat']
df[df['type'] == 'mtb'].groupby('groupset_cat').size().sort_values(ascending=False)

In [ ]:
df['speeds'] = df['Number of Speeds'].combine_first(df['Number of Gears'])
df['speeds_num'] = df['speeds'].str.extract(r'\b(\d{1,2})\b').astype(float)

def re_count_gears(x):
    if x < 5 or x > 33:
        return np.nan
    elif x <= 12:
        return x
    else:
        if x % 2 == 0:
            if x/2 > 12:
                return np.nan
            else:
                return x/2
        elif x % 3 == 0:
            return x/3

speeds_dict_num = {5: 0.2, 6: 0.3, 7: 0.4, 8: 0.5, 9: 0.6, 10: 0.7, 11: 0.8, 12: 0.9}
df['speeds_num'] = df['speeds_num'].apply(re_count_gears)

In [ ]:
speeds_road_dict_cat = {5: 'tourney', 6: 'tourney', 7: 'claris', 8: 'sora', 9: 'tiagra', 10: '105', 11: 'ultegra', 12: 'dura ace'}
speeds_mtb_dict_cat = {5: 'tourney', 6: 'tourney', 7: 'altus', 8: 'acera', 9: 'alivio', 10: 'deore', 11: 'deore xt', 12: 'xtr'}
def assign_group(row, gear_to_group_dict_road, gear_to_group_dict_mtb):
    bike_type, year, speed = row['type'], row['Production_year'], row['speeds_num']
    if pd.isna(speed):
        return np.nan
    gear_to_group_dict = gear_to_group_dict_road if bike_type == 'Road Bike' else gear_to_group_dict_mtb
    if pd.isna(year):
        return gear_to_group_dict['2017 2023'][speed]

    for time_peroid in gear_to_group_dict:
        start, end = map(float, time_peroid.split())
        if start <= year <= end:
            if speed in gear_to_group_dict[time_peroid]:
                return gear_to_group_dict[time_peroid][speed]
    return np.nan

df['speeds_cat'] = df.apply(assign_group, axis=1, args=(gear_to_group_dict_road, gear_to_group_dict_mtb, ))
df['groupset_cat'] = df['groupset_cat'].combine_first(df['speeds_cat'])
df['groupset_num'] = df['groupset_cat'].map(bike_groups_dict)

print_nans_perc(df['groupset_num'], p=True)

In [ ]:
materials = ['carbon', 'alu', 'aluminium', 'steel', 'alloy', 'titanium', 'chromoly']
materials_dict = {'carbon': 0.9, 'alu': 0.5, 'aluminium': 0.5, 'steel': 0.3, 'alloy': 0.5, 'titanium': 0.6, 'chromoly': 0.15}

cols_mt = ['Title', 'Seller notes', 'Model', 'Frame Material', 'Material', 'Fork Material', 'Rim Material']
df = make_num_cat(df, 'material', materials, materials_dict, find_cols=cols_mt)
df['material_cat']

In [ ]:
e_shift = ['di2', 'axs', 'etap']
e_shift_dict = {'di2': 1, 'axs': 1, 'etap': 1, np.nan: 0}

e_cols = ['Title', 'Seller notes', 'Model', 'Gear Change Mechanism', 'Features', 'Shifter Style']
df = make_num_cat(df, 'e_shift', e_shift, e_shift_dict, find_cols=e_cols)
df['e_shift_cat'].fillna('null', inplace=True)

In [ ]:
df = df[df['Condition'] != 'for parts or not working']
condition_dict = {'used': 0, 'new': 1, 'new other (see details)': 1, 'seller refurbished': 0, 'manufacturer refurbished': 0, '--': 0}

df['Condition'] = df['Condition'].combine_first(df['Item condition'])
df['isNew'] = df['Condition'].replace(condition_dict, regex=True)
df['isNew'] = df['isNew'].astype(float)
print_nans_perc(df['isNew'], p = True)
df.groupby('isNew').size().sort_values()

In [ ]:
brakes = ['rim', 'disc', 'v-brake', 'hydraulic', 'caliper', 'cantilever', 'v-brakes']
brakes_dict = {'rim': 0.5, 'disc': 1, 'v-brake': 0.5, 'hydraulic': 1, 'caliper': 0.5, 'cantilever': 0.5, 'v-brakes': 0.5}

cols_br = ['Title', 'Seller notes', 'Model', 'Brake Type', 'Braking Type']
df = make_num_cat(df, 'brake', brakes, brakes_dict, find_cols=cols_br)

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(2,2, figsize = (8,6))

df[df['groupset_cat'] == 'ultegra']['Production_year'].plot(kind='hist', bins=50, title='ultegra', ax=axs[0,0])
df[df['groupset_cat'] == '105']['Production_year'].plot(kind='hist', bins=50, title='105', ax=axs[0,1])
df[df['groupset_cat'] == 'dura ace']['Production_year'].plot(kind='hist', bins=50, title='dura ace', ax=axs[1,0])
df[df['groupset_cat'] == 'sram red']['Production_year'].plot(kind='hist', bins=50, title='sram red', ax=axs[1,1])
plt.tight_layout()
plt.show()

groupset_median = df.groupby('groupset_cat')['Production_year'].median()[df.groupby('groupset_cat').size() > 60]

In [ ]:
print_nans_perc(df['Production_year'], p = 1)
df.loc[df['groupset_cat'].isin(groupset_median.index), 'Production_year'] = df.loc[df['groupset_cat'].isin(groupset_median.index), 'Production_year'].fillna(df['groupset_cat'].map(groupset_median.to_dict()))
print_nans_perc(df['Production_year'], p = 1)

In [ ]:
from sklearn.preprocessing import QuantileTransformer

min = 1950
max = 2020
df['year_num'] = df['Production_year'].apply(lambda x: (x - min)/(max - min))

notna_idx = df.index[df['Production_year'].notna()]
arr = df.loc[notna_idx, 'Production_year'].to_numpy()
arr = arr.reshape(-1, 1)
qt = QuantileTransformer(n_quantiles=len(arr), output_distribution='uniform')
arr = qt.fit_transform(arr)
arr = arr.flatten()
df.loc[notna_idx,'year_num'] = arr

fig, axs = plt.subplots(1, 2, figsize=(8, 6)) 

df['Production_year'].plot(kind='hist', bins=50, ax=axs[0], title='years')
df['year_num'].plot(kind='hist', bins=50, ax=axs[1], title='years processed')

plt.tight_layout()
plt.show()
df['year_num']

In [ ]:
cols = print_nans_perc(df)
to_drop = ['Frame Size', 'Department', 'UPC', 'Gender', 'Shifter Style', 
              'Features', 'Country/Region of Manufacture', 'Tyre Type', 'Style', 'Manufacturer Color', 'Tire Type',
              'Modified Item', 'Tires', 'Custom-Built', 'Custom Bundle', 'Handlebar Type', 'Shifter', 'Gear Change Mechanism', 
              'Model Year', 'Configuration', 'Frame Material', 'Brake Type', 'Condition', 'Number of Gears', 'speeds', 'Number of Speeds', 
              'Material', 'speeds_num', 'Vintage', 'Model', 'Brand', 'speeds_cat', 'MPN','Size', 'Rear Derailleur', 'Braking Type', 
              'Groupset', 'Riding Style', 'Frame in inches', 'Size CM', 'Availability', 'Item condition', 'Price was', 'Price now', 'Type', 'Bike Type']
to_drop += list(cols)

df_drp = df.drop(to_drop, axis = 1, inplace=False)
print_nans_perc(df_drp, p=True)


In [ ]:
roadies = df_drp[df_drp['type'] == 'road bike'].reset_index(drop=1)

cols_to_drop_road = ['Wheel Size', 'Suspension', 'Suspension Type', 'Seller notes']
roadies.drop(columns=cols_to_drop_road, inplace=True)
print_nans_perc(roadies, p=1)

In [ ]:
old_idx = roadies.loc[roadies['Production_year'] < 2016].index
roadies.loc[old_idx, 'brake_num'] = 0.5
roadies.loc[old_idx, 'brake_cat'] = 'rim'
print_nans_perc(roadies['brake_num'], p = 1)

In [ ]:
mtb = df_drp[df_drp['type'].isin(['mtb', 'full', 'hard'])]

print_nans_perc(mtb, p=True)

In [ ]:
wheel_size = ['26', '27.5', '27', '27.5', '29', '650b', '24']
wheel_size_dict = {'27.5': 0.66, '29': 1, '27': 0.66, '27,5': 0.66, '26': 0.33, '650b': 0.66, '24': 0.25}

mtb = make_num_cat(mtb, 'wheel', wheel_size, wheel_size_dict)

In [ ]:
mtb['suspension'] = mtb['Suspension'].combine_first(mtb['Suspension Type'])
mtb.drop(columns=['Suspension', 'Suspension Type'], inplace=True)
mtb.loc[mtb['suspension'].isin(['full suspension (front & rear)','front & rear (full)']), 'suspension'] = 'full'

In [ ]:
susps = ['front', 'full', 'no suspension', 'rear', 'hard']
susps_dict = {'front': 0.5, 'full': 1, 'no suspension': 0, 'rear': 1, 'hard': 0.5}

sus_cols = ['suspension', 'type']
mtb = make_num_cat(mtb, 'susp', susps, susps_dict, find_cols=sus_cols)

In [ ]:
old_idx = mtb.loc[mtb['Production_year'] > 2010].index
mtb.loc[old_idx, 'brake_num'] = 1
mtb.loc[old_idx, 'brake_cat'] = 'disc'
print_nans_perc(mtb['brake_cat'], p=1)

In [ ]:
mtb_num = mtb.drop(columns=['groupset_cat', 'brake_cat', 'material_cat', 'type', 'Wheel Size', 'wheel_cat', 'Seller notes', 'Title', 'suspension', 'susp_cat', 'e_shift_cat', 'Production_year'])
mtb_cat = mtb.drop(columns=['groupset_num', 'brake_num', 'material_num', 'type', 'Wheel Size', 'wheel_num', 'Seller notes', 'Title', 'suspension', 'susp_num', 'Production_year'])
print_nans_perc(mtb_cat, p=True)

mtb_num.dropna(axis=0, how='any', inplace=True)
mtb_cat.dropna(axis=0, how='any', inplace=True)
num_cols = ['ID', 'Price','year_num', 'isNew', 'susp_num','material_num', 'groupset_num', 'e_shift_num', 'wheel_num', 'brake_num']
cat_cols = ['ID', 'Price','year_num', 'isNew', 'susp_cat','material_cat', 'groupset_cat', 'e_shift_cat', 'wheel_cat', 'brake_cat']
mtb_num = mtb_num[num_cols]
mtb_num.to_csv('data\\num_data\mtb_num.csv')

mtb_one_hot = pd.get_dummies(mtb_cat, columns=['susp_cat','material_cat', 'groupset_cat', 'e_shift_cat', 'wheel_cat', 'brake_cat'])
mtb_cat.to_csv('data\\cat_data\mtb_one_hot.csv')

print_nans_perc(roadies, p=True, sort_by='idx')
print_nans_perc(roadies.dropna(axis=0, how='any'), p=True, sort_by='idx')
roadies_num = roadies.drop(columns=['type', 'Title', 'groupset_cat', 'brake_cat', 'material_cat', 'e_shift_cat', 'Production_year'])
roadies_cat = roadies.drop(columns=['groupset_num', 'brake_num', 'material_num', 'type', 'Title', 'e_shift_num', 'Production_year'])
roadies_num.dropna(axis=0, how='any', inplace=True)
roadies_cat.dropna(axis=0, how='any', inplace=True)

roadies_num.to_csv('data\\num_data\\road_num.csv')

roadies_one_hot = pd.get_dummies(roadies_cat, columns=['groupset_cat', 'brake_cat', 'material_cat', 'e_shift_cat'])
roadies_one_hot.to_csv('data\\cat_data\\road_one_hot.csv')
